## 2. 키보드 구조 & 위치 정의

### 2.1 기본 배치 (변경 전)
```
행 0:  1(ㅂ)  2(ㅈ)  3(ㄷ)  4(ㄱ)  5(ㅅ)  6(ㅛ)  7(ㅕ)  8(ㅑ)  9(ㅐ) 10(ㅔ)
행 1: 11(ㅁ) 12(ㄴ) 13(ㅇ) 14(ㄹ) 15(ㅎ) 16(ㅗ) 17(ㅓ) 18(ㅏ) 19(ㅣ)   -
행 2: 20(ㅋ) 21(ㅌ) 22(ㅊ) 23(ㅍ) 24(ㅠ) 25(ㅜ) 26(ㅡ)   -     -     -
```

### 2.2 위치 인덱스 매핑
```python
# position_index (0~29) ↔ (row, col, hand, finger)

# 왼손 (L)
#   손가락 순서: ring(약), middle(중), index(검), little(소)
positions = {
    0: (0, 0, 'L', 'ring'),     # 위치 0: 행0열0, 왼손 약지
    1: (0, 1, 'L', 'middle'),
    2: (0, 2, 'L', 'index'),
    3: (0, 3, 'L', 'index'),   # 같은 줄 index 여러 개
    4: (0, 4, 'L', 'little'),  # 또는 pinky로 표기
    
    10: (1, 0, 'L', 'ring'),
    11: (1, 1, 'L', 'middle'),
    12: (1, 2, 'L', 'index'),
    13: (1, 3, 'L', 'index'),
    14: (1, 4, 'L', 'little'),
    
    20: (2, 0, 'L', 'ring'),
    21: (2, 1, 'L', 'middle'),
    22: (2, 2, 'L', 'index'),
    23: (2, 3, 'L', 'little'),
    
    # 오른손 (R)
    5: (0, 5, 'R', 'index'),
    6: (0, 6, 'R', 'index'),
    7: (0, 7, 'R', 'middle'),
    8: (0, 8, 'R', 'ring'),
    9: (0, 9, 'R', 'little'),
    
    15: (1, 5, 'R', 'index'),
    16: (1, 6, 'R', 'index'),
    17: (1, 7, 'R', 'middle'),
    18: (1, 8, 'R', 'ring'),
    19: (1, 9, 'R', 'little'),
    
    24: (2, 4, 'R', 'index'),
    25: (2, 5, 'R', 'middle'),
    26: (2, 6, 'R', 'ring'),
}

# 26~29: 미사용 위치 (패딩)
```

### 2.3 레이아웃 표현
```python
layout = np.array([
    [0, 1, 2, 3, 4, 5, 6, 7, 8, 9],           # 행 0: 10글자
    [10, 11, 12, 13, 14, 15, 16, 17, 18, -1], # 행 1: 9글자 + 1빈칸
    [20, 21, 22, 23, 24, 25, 26, -1, -1, -1]  # 행 2: 7글자 + 3빈칸
])

# layout[r, c] = 글자_인덱스 (0~25)
# -1 = 빈 위치

# 예: 글자 0(ㅂ)의 위치 찾기
row, col = np.where(layout == 0)  # row=[0], col=[0]
position_idx = row[0] * 10 + col[0]  # = 0
(r, c, hand, finger) = position_table[position_idx]  # (0, 0, 'L', 'ring')
```

## 3. 거리 계산 (f1 기초)

### 3.1 거리 공식
```
거리 = sqrt((col_j - col_i)^2 + weight * (row_j - row_i)^2)

여기서 weight는 "행 변화"를 상대적으로 중요하게 봄
예: weight = 1.0 또는 0.8
```

### 3.2 현재 코드 문제
```python
# models/keyboard_layout.py의 distance() 메서드
def distance(self, pos1, pos2):
    (x1, y1) = self.keyboard.keyboard[pos1]  # ← 오류!
    # self.keyboard.keyboard는 2D 배열이 아니라
    # self.keyboard 자체가 2D 배열 (이미 keyboard_table)
```

### 3.3 올바른 구현
```python
def get_position_2d(layout, char_idx):
    """글자 인덱스 → (row, col) 반환"""
    rows, cols = np.where(layout == char_idx)
    if len(rows) > 0:
        return (rows[0], cols[0])
    return None

def distance(pos1, pos2, weight=0.8):
    """두 위치 (row, col) 사이의 거리"""
    r1, c1 = pos1
    r2, c2 = pos2
    return np.sqrt((c2 - c1)**2 + weight * (r2 - r1)**2)

# 사용 예
layout = ...  # (3, 10) 배열
pos_i = get_position_2d(layout, char_i)
pos_j = get_position_2d(layout, char_j)
dist_ij = distance(pos_i, pos_j)
```

## 4. 손가락 정보 (f2 기초)

### 4.1 f2 테이블
```
손가락 종류별 피로 지수:
  Index:  1.0  (검지 - 가장 강함)
  Middle: 1.0  (중지 - 검지와 동등)
  Ring:   1.2  (약지 - 약간 약함)
  Little: 1.5  (소지 - 가장 약함)
```

### 4.2 두 글자 사이의 f2 비용
```python
def get_finger_cost(pos1, pos2, position_table, f2_table):
    """
    두 글자 사이의 손가락 비용
    보통: 평균 사용
    """
    _, _, _, finger1 = position_table[get_position_idx(pos1)]
    _, _, _, finger2 = position_table[get_position_idx(pos2)]
    
    f2_1 = f2_table[finger1]  # 1.0, 1.0, 1.2, 1.5
    f2_2 = f2_table[finger2]
    
    return (f2_1 + f2_2) / 2

# 예
# 검지(1.0) → 약지(1.2) → f2_cost = (1.0 + 1.2) / 2 = 1.1
```

## 5. 방향 정보 (f3 기초)

### 5.1 f3 테이블
```
입력 방향에 따른 피로도:

같은 손(Same Hand):
  위→아래 (Top→Bottom):  1.2  (손가락 늘려야 함)
  아래→위 (Bottom→Top):  1.0  (손가락 오므려 편함)
  같은 줄 (Same Row):    1.0  (옆으로 슬라이드)

다른 손(Different Hand):
  위→아래 (Top→Bottom):  1.5  (손 전환 + 늘리기)
  아래→위 (Bottom→Top):  1.2  (손 전환 + 오므리기)
  같은 줄 (Same Row):    1.0  (손 전환만)
```

### 5.2 방향 판단 로직
```python
def get_direction_cost(pos_i, pos_j, position_table, f3_table):
    """
    i → j 입력 시 방향 비용
    """
    row_i, col_i, hand_i, _ = position_table[get_position_idx(pos_i)]
    row_j, col_j, hand_j, _ = position_table[get_position_idx(pos_j)]
    
    # 1. 손 동일 여부
    same_hand = (hand_i == hand_j)
    
    # 2. 행 변화
    if row_j > row_i:
        row_direction = "top_to_bottom"
    elif row_j < row_i:
        row_direction = "bottom_to_top"
    else:
        row_direction = "same_row"
    
    # 3. 테이블 조회
    if same_hand:
        key = ("same_hand", row_direction)
    else:
        key = ("diff_hand", row_direction)
    
    return f3_table[key]

# 예
# pos_i = (0, 0, 'L'), pos_j = (1, 1, 'L')
# → row 증가 (위→아래), 손 같음
# → f3_cost = f3_table["same_hand", "top_to_bottom"] = 1.2
```

## 6. 손가락 조합 (f4 기초)

### 6.1 f4 테이블
```
손가락 조합 비용 (4×4 대칭 행렬):

            Index  Middle  Ring   Little
Index       2.0    1.0    1.2    1.0
Middle      1.0    2.0    1.5    1.2
Ring        1.2    1.5    2.0    1.5
Little      1.0    1.2    1.5    2.0

의미:
- 대각선(같은 손가락 연속): 2.0 (힘들다)
- 인접한 손가락: 1.0~1.5 (적당)
- 먼 손가락: 1.0~1.2 (편함)
```

### 6.2 손가락조합 비용 계산
```python
def get_combination_cost(pos_i, pos_j, position_table, f4_table):
    """
    i에서 j로 입력할 때 손가락 조합 피로도
    """
    _, _, _, finger_i = position_table[get_position_idx(pos_i)]
    _, _, _, finger_j = position_table[get_position_idx(pos_j)]
    
    return f4_table[finger_i, finger_j]

# 예
# i를 검지로, j를 약지로 누름
# f4_cost = f4_table['Index', 'Ring'] = 1.2
```

## 7. 통합: 전체 피로도 계산

### 7.1 수식
$$C_{\text{fatigue}}(M) = \sum_{i=0}^{25} \sum_{j=0}^{25} W[i,j] \times f_{\text{step}}(i,j,M)$$

$$f_{\text{step}}(i,j,M) = d(i,j) \times f_2(i,j) \times f_3(i,j) \times f_4(i,j)$$

### 7.2 구현 의사코드
```python
def evaluate_fatigue(layout, W, position_table, 
                    f2_table, f3_table, f4_table):
    """
    레이아웃의 총 피로도 계산
    
    Args:
        layout: (3, 10) 배열, layout[r,c] = 글자인덱스
        W: (26, 26) 공기 행렬
        position_table: dict, pos_idx → (row, col, hand, finger)
        f2_table, f3_table, f4_table: 피로도 테이블들
    
    Returns:
        float: 총 피로도 (작을수록 좋음)
    """
    C_fatigue = 0.0
    
    # 모든 글자쌍 순회
    for i in range(26):
        for j in range(26):
            if W[i, j] > 0:  # 공기도가 0보다 크면
                # 위치 찾기
                pos_i = get_position_2d(layout, i)
                pos_j = get_position_2d(layout, j)
                
                if pos_i is None or pos_j is None:
                    continue  # 레이아웃에 없으면 스킵
                
                # 거리
                d = distance(pos_i, pos_j, weight=0.8)
                
                # f2, f3, f4
                f2 = get_finger_cost(pos_i, pos_j, position_table, f2_table)
                f3 = get_direction_cost(pos_i, pos_j, position_table, f3_table)
                f4 = get_combination_cost(pos_i, pos_j, position_table, f4_table)
                
                # f_step
                f_step = d * f2 * f3 * f4
                
                # 누적
                C_fatigue += W[i, j] * f_step
    
    return C_fatigue
```

### 7.3 라플라시안 보조항
```python
def evaluate_laplacian_penalty(layout, W, weight=0.3):
    """
    라플라시안 기반 페널티:
    자주 함께 쓰이는 글자(W[i,j]크면)가 멀리 떨어져있으면 페널티
    """
    C_lap = 0.0
    
    for i in range(26):
        for j in range(26):
            if W[i, j] > 0:
                pos_i = get_position_2d(layout, i)
                pos_j = get_position_2d(layout, j)
                
                if pos_i is None or pos_j is None:
                    continue
                
                # 기울기 제곱 거리 (거리의 영향도 고려)
                dist_sq = ((pos_i[0] - pos_j[0])**2 + 
                          (pos_i[1] - pos_j[1])**2)
                
                C_lap += W[i, j] * dist_sq
    
    return weight * C_lap
```

### 7.4 최종 비용 & 적합도
```python
def evaluate_individual(layout, W, position_table,
                       f2_table, f3_table, f4_table,
                       lap_weight=0.3):
    """
    개체의 전체 적합도 평가
    """
    # 피로도 + 라플라시안 페널티
    C_fatigue = evaluate_fatigue(layout, W, position_table,
                                f2_table, f3_table, f4_table)
    C_lap = evaluate_laplacian_penalty(layout, W, weight=lap_weight)
    
    C_total = C_fatigue + C_lap
    
    # 적합도 (비용 작을수록 적합도 큼)
    Fitness = 1.0 / (C_total + 1e-6)
    
    return Fitness, {
        'C_fatigue': C_fatigue,
        'C_lap': C_lap,
        'C_total': C_total,
        'Fitness': Fitness
    }
```

## 8. 요약: 평가 구조도

```
입력: layout (3×10 배열)
  ↓
[Step 1] 글자 위치 추출
  for i in 26글자:
    pos_i = get_position_2d(layout, i)
  ↓
[Step 2] 모든 글자쌍 순회
  for i, j in 0~25:
    if W[i,j] > 0:
      ↓
      [Step 2-1] 거리 계산
        d_ij = sqrt((c_j-c_i)² + 0.8*(r_j-r_i)²)
      ↓
      [Step 2-2] 손가락 비용
        f2_ij = (f2[finger_i] + f2[finger_j]) / 2
      ↓
      [Step 2-3] 방향 비용
        f3_ij = f3_table[hand_match, row_direction]
      ↓
      [Step 2-4] 손가락조합 비용
        f4_ij = f4_table[finger_i, finger_j]
      ↓
      [Step 2-5] f_step 계산
        f_step = d_ij × f2_ij × f3_ij × f4_ij
      ↓
      [Step 2-6] 가중합
        C_fatigue += W[i,j] × f_step
  ↓
[Step 3] 라플라시안 페널티
  for i, j in 0~25:
    if W[i,j] > 0:
      dist_sq = (r_j - r_i)² + (c_j - c_i)²
      C_lap += W[i,j] × dist_sq
  ↓
[Step 4] 최종 비용
  C_total = C_fatigue + 0.3 × C_lap
  ↓
[Step 5] 적합도
  Fitness = 1 / (C_total + ε)
  ↓
출력: Fitness (높을수록 좋음)
```

In [ ]:
# 실제 예시 계산

import numpy as np

# 1. 테이블 정의
f2_table = {
    'Index': 1.0,
    'Middle': 1.0,
    'Ring': 1.2,
    'Little': 1.5
}

f3_table = {
    ('same_hand', 'top_to_bottom'): 1.2,
    ('same_hand', 'bottom_to_top'): 1.0,
    ('same_hand', 'same_row'): 1.0,
    ('diff_hand', 'top_to_bottom'): 1.5,
    ('diff_hand', 'bottom_to_top'): 1.2,
    ('diff_hand', 'same_row'): 1.0,
}

f4_table = np.array([
    [2.0, 1.0, 1.2, 1.0],  # Index
    [1.0, 2.0, 1.5, 1.2],  # Middle
    [1.2, 1.5, 2.0, 1.5],  # Ring
    [1.0, 1.2, 1.5, 2.0],  # Little
])

# 2. 위치 테이블 (간략)
position_table = {
    0: (0, 0, 'L', 'ring'),
    1: (0, 1, 'L', 'middle'),
    2: (0, 2, 'L', 'index'),
    3: (0, 3, 'L', 'index'),
    5: (0, 5, 'R', 'index'),
    6: (0, 6, 'R', 'index'),
}

finger_map = {
    'Index': 0,
    'Middle': 1,
    'Ring': 2,
    'Little': 3
}

# 3. 거리 계산 예시
pos_i = (0, 0)  # row 0, col 0
pos_j = (0, 1)  # row 0, col 1
distance = np.sqrt((pos_j[1] - pos_i[1])**2 + 0.8 * (pos_j[0] - pos_i[0])**2)
print(f"위치 (0,0) → (0,1) 거리: {distance:.4f}")

# 4. f2 계산 예시
finger_i_name, finger_j_name = 'ring', 'middle'
f2_cost = (f2_table[finger_i_name.capitalize()] + f2_table[finger_j_name.capitalize()]) / 2
print(f"손가락 {finger_i_name} → {finger_j_name}, f2 비용: {f2_cost:.4f}")

# 5. f3 계산 예시
hand_same = True
row_direction = 'same_row'
f3_cost = f3_table[('same_hand' if hand_same else 'diff_hand', row_direction)]
print(f"같은 손, 같은 줄, f3 비용: {f3_cost:.4f}")

# 6. f4 계산 예시
finger_i_idx = finger_map['Ring']
finger_j_idx = finger_map['Middle']
f4_cost = f4_table[finger_i_idx, finger_j_idx]
print(f"손가락 Ring → Middle, f4 비용: {f4_cost:.4f}")

# 7. f_step 계산
f_step = distance * f2_cost * f3_cost * f4_cost
print(f"\n전체 f_step = {distance:.4f} × {f2_cost:.4f} × {f3_cost:.4f} × {f4_cost:.4f} = {f_step:.4f}")

# 8. 공기도 가중치
W_example = 1000  # W[i,j] = 1000 (자주 함께 나타남)
contribution = W_example * f_step
print(f"W[i,j] × f_step = {W_example} × {f_step:.4f} = {contribution:.2f}")

## 9. 다음 단계: 구현

### 수정 순서
1. **keyboard_layout.py 수정**
   - position_table 정의
   - distance() 메서드 수정
   - get_position(), get_hand(), get_finger() 추가

2. **fatigue.py 수정**
   - f2_table, f3_table, f4_table 정의
   - evaluate_fatigue() 함수 구현
   - evaluate_laplacian_penalty() 함수 구현

3. **ga_integrated.py 수정**
   - Individual.evaluate() 완전 재구현
   - 모든 26×26 글자쌍 순회
   - 정확한 f_step 계산

4. **테스트**
   - 단위 테스트 (각 함수별)
   - 통합 테스트 (전체 GA)
   - 결과 시각화

### 예상 효과
- ✓ 평가 함수 정확화 → GA 수렴성 개선
- ✓ 피로도 모델 통합 → 현실성 있는 배열
- ✓ 모든 데이터 활용 → 신뢰할 수 있는 결과